In [1]:
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB

In [2]:
def MaxMinDiversity(subset: np.ndarray, complete: np.ndarray) -> float:
    """
    Calculate Max-Min Diversity, the minimum distance between all pairs of points in the given subset.
    
    Parameters:
        subset: Subset of points
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_dist: Max-Min Diversity, the minimum distance between all pairs of points in the subset
    """
    if len(subset) < 2:
        return 0
    
    n = len(subset)
    min_dist = float('inf')
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = complete[subset[i]][subset[j]]
            min_dist = min(min_dist, dist)
    
    return min_dist

In [3]:
def FairRadius(k: int, complete: np.ndarray) -> np.ndarray:
    """
    Calculate Fair Radius for all points in the dataset.

    Parameters:
        k: Integer parameter k, used to calculate minimum number of points each point needs to cover ⌈n/k⌉
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        fair_radius: Fair Radius for all points in the dataset
    """
    n = complete.shape[0]
    min_points = int(np.ceil(n / k))
    fair_radius = []

    for i in range(n):
        distances = complete[i]
        sorted_distances = np.sort(distances)
        point_radius = sorted_distances[min_points-1]
        fair_radius.append(point_radius)
        
    return np.array(fair_radius)


In [4]:
def CriticalRegion(alpha: float, fair_radius: np.ndarray, complete: np.ndarray) -> tuple[np.ndarray, dict]:
    """
    Generate the Critical Regions by finding a set of centers that cover all points.
    
    Parameters:
        alpha: Fairness parameter
        fair_radius: Array containing fair radius values for all points
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping each center to its covered points
    """
    covered_points = set()
    n = len(fair_radius)
    points = set(range(n))
    selected_centers = []
    critical_regions = {}

    while covered_points != points:
        minus = points - covered_points
        c = min(minus, key=lambda x: fair_radius[x])
        selected_centers.append(c)
        for point in minus:
            if complete[point][c] <= 2 * alpha * fair_radius[point]:
                covered_points.add(point)
    selected_centers = np.array(selected_centers)

    for center in selected_centers:
        r_c = fair_radius[center]
        distances = complete[center]
        points_in_circle = np.array([i for i, dist in enumerate(distances) if dist <= alpha * r_c])
        critical_regions[center] = points_in_circle

    return selected_centers, critical_regions


In [5]:
def IFDM(k: int, complete: np.ndarray, epsilon: float, beta: float, selected_centers: np.ndarray, critical_regions: dict) -> tuple[np.ndarray, tuple[tuple[np.ndarray, int], ...], np.ndarray, dict]:
    """
    Construct an instance of k-clustering under partition matroid constraint corresponding to the given instance of alpha-fair k-clustering.
    
    Parameters:
        k: Target number of clusters
        alpha: Fairness parameter
        complete: Complete graph adjacency matrix with distances
        epsilon: Accuracy parameter (< 1/2)
        beta: Approximation guarantee parameter (≤ 1)
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping centers to covered points
        
    Returns:
        P_prime: Augmented point set with duplicated points
        centers_info: Tuple containing center selection info
        d_prime: Modified graph adjacency matrix with distances
        corresponding: The dictionary for getting the corresponding point in originial data
    """
    n = complete.shape[0]
    P_0 = list(range(n))
    P_1 = list(range(n))
    current_max_id = n
    
    B_copies = {}
    corresponding = {}
    for center in selected_centers:
        copies = []
        for point in critical_regions[center]:
            corresponding[current_max_id] = point
            copies.append(current_max_id)
            P_1.append(point)
            current_max_id += 1
        B_copies[center] = np.array(copies)

    P_prime = np.arange(current_max_id)
    
    k_i = {center: 1 for center in selected_centers}
    
    k_0 = k - len(selected_centers)

    delta = np.min(complete[complete > 0])

    n_prime = len(P_prime)
    d_prime = np.zeros((n_prime, n_prime))

    for i in range(n_prime):
        for j in range(n_prime):
            if i == j:
                d_prime[i][j] = 0
            elif P_1[i] != P_1[j]:
                d_prime[i][j] = round(complete[P_1[i]][P_1[j]], 10)
            else:
                d_prime[i][j] = round(epsilon * beta * delta, 10)
                
    return P_prime, ((P_0, k_0), *((B_copies[c], k_i[c]) for c in selected_centers)), d_prime, corresponding


In [6]:
def distancePS(centerSet: np.ndarray, i: int, complete: np.ndarray) -> float:
    """
    Returns the distance between a certain point and a certain set.
    
    Parameters:
        centerSet: A numpy array containing confirmed center indexes
        i: The index of any point
        complete : Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_distance: The distance between point and center set
    """
    min_distance = float("inf")
    for center in centerSet:
        distance = complete[center][i]
        if (distance < min_distance):
            min_distance = distance
    
    return min_distance

In [7]:
def GMM(points_index: np.ndarray, k: int, complete: np.ndarray) -> np.ndarray:
    """
    Returns indexes of k centers after running GMM Algorithm.
    
    Parameters: 
        points_index: The indexes of data
        k: A decimal integer, the number of centers
        complete: Complete graph adjacency matrix containing distances between all pairs of points
        initial: An initial set of elements
    
    Returns:
        centers: A numpy array with k indexes as center point indexes
    """
    centers = []
    initial_point_index = random.choice(points_index)
    centers.append(initial_point_index)
    while (len(centers) < k):
        max_distance = 0
        max_distance_vector_index = None
        for i in points_index:
            distance = distancePS(centers, i, complete)
            if distance > max_distance:
                max_distance = distance
                max_distance_vector_index = i
        centers.append(max_distance_vector_index)
    centers = np.array(centers)

    return centers

In [8]:
def ILP(n, E, k, color_constraints, color_sets):
    """
    Returns an ILP solution for FMMD-S.
    
    Parameters:
        n: The number of points
        E: The undirected graph generated by FMMD-S
        k: A decimal integer, the number of centers 
        color_constraints: The groups that the data are divided into
        color_sets: The points in different groups
    
    Returns:
        selected_nodes: A numpy array containing selected elements that maximize the minimum pairwise distance    
    """
    model = gp.Model("ILP")
    model.setParam('OutputFlag', 0)
    print(n)
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)

    print(E)
    eid = 0
    for e in E:
        model.addConstr(x[e[0]] + x[e[1]] <= 1, f"edge_{str(eid)}")
        eid += 1

    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= k, "max_selection")

    for c, number in color_constraints.items():
        nodes_in_color = color_sets[c]
        if c == 0:
            print("It was triggered.")
            model.addConstr(gp.quicksum(x[i] for i in nodes_in_color) <= number, f"number_color_{c}")
        else:
            print(nodes_in_color)
            model.addConstr(gp.quicksum(x[i] for i in nodes_in_color) == number, f"number_color_{c}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        selected_nodes = np.array([i for i in range(n) if x[i].x > 0.5])
        return selected_nodes
    else:
        raise ValueError("No optimal solution found.")
        

In [9]:
def FMMDS(sets: tuple, k: int, error: float, complete: np.ndarray) -> np.ndarray:
    """
    Returns a subset with high Max-min diversity under partition matroid constraint.
    
    Parameters:
        sets: A tuple containing partitioned sets returned by IFDM function
        k: A decimal integer, the number of elements to be selected
        error: A float number indicating the error parameter
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        solution: A numpy array containing selected elements that maximize the minimum pairwise distance
    """
    amount = complete.shape[0]
    complete_array = np.arange(amount)
    U_gmm = GMM(complete_array, k, complete)
    category_number = len(sets)
    U_c = []
    for i in range(category_number):
        intersection = np.intersect1d(sets[i][0], U_gmm)
        if len(intersection) > 0:
            U_c.append(intersection)
        else:
            random_element = np.random.choice(sets[i][0])
            U_c.append(np.array([random_element]))
            
    distance_p = 2 * MaxMinDiversity(U_gmm, complete)

    S = []
    while len(S) == 0:
        for c in range(category_number):  
            max_div_uc_v = 0
            for v in sets[c][0]:
                new_div_uc_v = MaxMinDiversity(np.union1d(U_c[c], [v]), complete)
                if new_div_uc_v > max_div_uc_v:
                    max_div_uc_v = new_div_uc_v
            while len(U_c[c]) < k and max_div_uc_v >= distance_p:
                max_distance = 0
                max_distance_vector_index = 0
                for i in sets[c][0]:
                    distance = distancePS(U_c[c], i, complete)
                    if distance > max_distance:
                        max_distance = distance
                        max_distance_vector_index = i
                U_c[c] = np.append(U_c[c], max_distance_vector_index)

        V_p = np.concatenate(U_c)
        index2V_p = dict()
        V_p2index = dict()
        for index, point in enumerate(V_p):
            index2V_p[index] = point
            V_p2index[point] = index

        E = []
        for i in range(len(V_p)):
             for j in range(i + 1, len(V_p)):
                  if complete[V_p[i]][V_p[j]] < distance_p / 2:
                       E.append((i, j))

        color_constraints = dict()
        color_sets = dict()
        for group in range(category_number):
            color_constraints[group] = sets[group][1]
            color_sets_original = np.intersect1d(sets[group][0], V_p)
            color_sets_ilp = []
            for point in color_sets_original:
                color_sets_ilp.append(V_p2index[point])
            color_sets[group] = np.array(color_sets_ilp)
        
        print(color_constraints)
            

        solution = ILP(len(V_p), E, k, color_constraints, color_sets)
        if len(solution) < k:
             distance_p = (1 - error) * distance_p
        else:
             S = solution

    for i in range(len(S)):
        S[i] = index2V_p[S[i]]
    
    return S

In [10]:
def SOL(S: np.ndarray, corresponding: dict, complete: np.ndarray) -> np.ndarray:
    """
    Returns a np.ndarray with high Max-min diversity in the original data.
    
    Parameters:
        S: The solution from FMMDS
        corresponding: The dictionary for getting the corresponding point in originial data
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        solution: A numpy array containing selected elements that maximize the minimum pairwise distance
    """
    solution = []
    for point in S:
        if point < complete.shape[0]:
            solution.append(point)
        elif corresponding[point] not in solution:
            solution.append(corresponding[point])
        else:
            continue
    solution = np.array(solution)

    return solution

In [11]:
def Examine(complete: np.ndarray, solution: np.ndarray, fair_radius: np.ndarray, alpha: float, gamma: float) -> bool:
    
    points = np.arange(complete.shape[0])
    for point in points:
        if distancePS(solution, point, complete) > alpha * gamma * fair_radius[point]:
            print(distancePS(solution, point, complete))
            print(alpha * gamma * fair_radius[point])
            return False
    
    return True

In [12]:
complete = np.load("dataset/adult_complete.npy")
complete

array([[0.        , 0.03521865, 0.0360143 , ..., 0.03676195, 0.03358565,
        0.037072  ],
       [0.03521865, 0.        , 0.03164265, ..., 0.03761507, 0.02821316,
        0.03421197],
       [0.0360143 , 0.03164265, 0.        , ..., 0.02900964, 0.00482017,
        0.0067394 ],
       ...,
       [0.03676195, 0.03761507, 0.02900964, ..., 0.        , 0.02602977,
        0.03397402],
       [0.03358565, 0.02821316, 0.00482017, ..., 0.02602977, 0.        ,
        0.00952895],
       [0.037072  , 0.03421197, 0.0067394 , ..., 0.03397402, 0.00952895,
        0.        ]])

In [ ]:
alpha = 1
for K in range(2, 21):
    b = GMM(np.arange(1000), K, complete)
    fairradius = FairRadius(K, complete)
    # fairradius
    critical = CriticalRegion(alpha, fairradius, complete)
    ifdm = IFDM(K, complete, 0.1, 0.95 / 5, critical[0], critical[1])
    solution = FMMDS(ifdm[1], K, 0.05, ifdm[2])
    real_sol = SOL(solution, ifdm[3], complete)
    print("The ratio: ", MaxMinDiversity(real_sol, complete) / MaxMinDiversity(b, complete))
    if Examine(complete, real_sol, fairradius, alpha, 3) == False:
        print(ifdm)
        raise ValueError("Pity...")


{0: 13, 1: 1, 2: 1}
Set parameter Username
Set parameter LicenseID to value 2625788
Academic license - for non-commercial use only - expires 2026-02-20
17
[(3, 15), (11, 15), (11, 16), (13, 15), (13, 16), (14, 15), (14, 16), (15, 16)]
It was triggered.
[15]
[16]


ValueError: No optimal solution found.

In [189]:
K

15

In [190]:
critical

(array([256, 594]),
 {256: array([  2,  16,  60,  61,  89,  99, 140, 152, 158, 196, 197, 205, 219,
         228, 248, 256, 280, 286, 297, 299, 306, 345, 363, 366, 393, 422,
         447, 474, 480, 489, 503, 505, 509, 536, 554, 557, 576, 578, 607,
         623, 627, 628, 659, 663, 670, 709, 723, 744, 787, 792, 820, 831,
         840, 852, 867, 871, 885, 893, 896, 902, 913, 929, 953, 969, 980,
         998, 999]),
  594: array([ 31,  47,  53,  87,  96, 105, 150, 171, 182, 185, 189, 216, 253,
         312, 316, 328, 338, 350, 372, 377, 385, 405, 406, 421, 432, 460,
         470, 472, 482, 488, 494, 495, 501, 507, 512, 518, 533, 573, 590,
         594, 604, 605, 610, 626, 647, 655, 694, 695, 699, 716, 738, 777,
         786, 795, 827, 848, 850, 854, 865, 888, 931, 938, 944, 950, 960,
         963, 982])})

In [199]:
ifdm[3]

{1000: 2,
 1001: 16,
 1002: 60,
 1003: 61,
 1004: 89,
 1005: 99,
 1006: 140,
 1007: 152,
 1008: 158,
 1009: 196,
 1010: 197,
 1011: 205,
 1012: 219,
 1013: 228,
 1014: 248,
 1015: 256,
 1016: 280,
 1017: 286,
 1018: 297,
 1019: 299,
 1020: 306,
 1021: 345,
 1022: 363,
 1023: 366,
 1024: 393,
 1025: 422,
 1026: 447,
 1027: 474,
 1028: 480,
 1029: 489,
 1030: 503,
 1031: 505,
 1032: 509,
 1033: 536,
 1034: 554,
 1035: 557,
 1036: 576,
 1037: 578,
 1038: 607,
 1039: 623,
 1040: 627,
 1041: 628,
 1042: 659,
 1043: 663,
 1044: 670,
 1045: 709,
 1046: 723,
 1047: 744,
 1048: 787,
 1049: 792,
 1050: 820,
 1051: 831,
 1052: 840,
 1053: 852,
 1054: 867,
 1055: 871,
 1056: 885,
 1057: 893,
 1058: 896,
 1059: 902,
 1060: 913,
 1061: 929,
 1062: 953,
 1063: 969,
 1064: 980,
 1065: 998,
 1066: 999,
 1067: 31,
 1068: 47,
 1069: 53,
 1070: 87,
 1071: 96,
 1072: 105,
 1073: 150,
 1074: 171,
 1075: 182,
 1076: 185,
 1077: 189,
 1078: 216,
 1079: 253,
 1080: 312,
 1081: 316,
 1082: 328,
 1083: 338,
 108

In [27]:
# for i in range(complete.shape[0]):
#     if distancePS(critical[0], i, complete) > 2 * alpha * fairradius[i]:
#         print(False)

In [28]:
# set(critical[1][critical[0][0]]) & set(critical[1][critical[0][1]])

In [29]:
# for i in range(1150):
#     for j in range(1150):
#         if ifdm[2][i][j] != ifdm[2][j][i]:
#             print(False)
#             break

In [30]:
# for i in range(1150):
#     for j in range(1150):
#         if ifdm[2][i][j] < 0:
#             print(False)
#             break
#         if ifdm[2][i][j] == 0:
#             if i != j:
#                 print(False)
#                 break

In [31]:
# for i in range(1150):
#     for j in range(1150):
#         for k in range(1150):
#             if ifdm[2][i][j] > ifdm[2][i][k] + ifdm[2][j][k] + 1e-8:
#                 print(False)
#                 print(ifdm[2][i][j], ifdm[2][i][k] + ifdm[2][j][k])

In [135]:
np.ceil(10 / 5) == 2

True